In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from matplotlib.patches import Rectangle
import json

# Utiliser l'objet topology_data chargé précédemment
print("🔍 Exploration de l'objet NetworkTopologyGraph...")

try:
    # Méthode 1: Essayer d'obtenir le graphe NetworkX
    print("📊 Extraction du graphe NetworkX...")
    
    if hasattr(topology_data, 'get_graph'):
        network_graph = topology_data.get_graph()
        print(f"✅ Graphe extrait via get_graph()")
        print(f"Type du graphe: {type(network_graph)}")
        
        if hasattr(network_graph, 'nodes') and hasattr(network_graph, 'edges'):
            print(f"🔗 Graphe NetworkX détecté!")
            print(f"   📊 Nœuds: {len(network_graph.nodes())}")
            print(f"   📊 Arêtes: {len(network_graph.edges())}")
        else:
            print(f"⚠️ get_graph() ne retourne pas un graphe NetworkX standard")
            network_graph = None
    else:
        print("❌ Méthode get_graph() non trouvée")
        network_graph = None
    
    # Méthode 2: Essayer d'obtenir les statistiques
    print(f"\n📈 Extraction des statistiques...")
    if hasattr(topology_data, 'get_graph_stats'):
        try:
            stats = topology_data.get_graph_stats()
            print(f"✅ Statistiques obtenues:")
            print(f"Type des stats: {type(stats)}")
            print(f"Contenu: {stats}")
        except Exception as e:
            print(f"❌ Erreur lors de l'obtention des stats: {e}")
    
    # Méthode 3: Explorer les autres attributs
    print(f"\n🔍 Exploration des attributs...")
    
    # Tester pending_requests
    if hasattr(topology_data, 'pending_requests'):
        pending = topology_data.pending_requests
        print(f"📝 Pending requests: {type(pending)} - Taille: {len(pending) if hasattr(pending, '__len__') else 'N/A'}")
    
    # Essayer find_node avec des exemples
    if hasattr(topology_data, 'find_node'):
        print(f"🔎 Méthode find_node disponible")
    
    # Essayer find_edge avec des exemples
    if hasattr(topology_data, 'find_edge'):
        print(f"🔗 Méthode find_edge disponible")
        
    # Si on a réussi à extraire un graphe NetworkX, le visualiser
    if network_graph and hasattr(network_graph, 'nodes') and hasattr(network_graph, 'edges'):
        print(f"\n🎨 CRÉATION DE LA VISUALISATION...")
        
        # Créer une visualisation complète
        fig, axes = plt.subplots(2, 2, figsize=(18, 14))
        fig.suptitle('Analyse de la Topologie du Réseau', fontsize=16, fontweight='bold')
        
        # 1. Graphique principal - Topologie complète
        ax1 = axes[0, 0]
        
        # Calculer les positions des nœuds
        print("🔄 Calcul des positions des nœuds...")
        if len(network_graph.nodes()) < 100:
            pos = nx.spring_layout(network_graph, k=3, iterations=100, seed=42)
        elif len(network_graph.nodes()) < 500:
            pos = nx.spring_layout(network_graph, k=2, iterations=50, seed=42)
        else:
            pos = nx.random_layout(network_graph, seed=42)
        
        # Colorier les nœuds selon leur degré
        degrees = dict(network_graph.degree())
        node_colors = [degrees[node] for node in network_graph.nodes()]
        node_sizes = [min(300, max(50, degrees[node] * 20)) for node in network_graph.nodes()]
        
        # Dessiner le graphe
        nx.draw_networkx_nodes(network_graph, pos, ax=ax1,
                              node_color=node_colors, 
                              node_size=node_sizes,
                              cmap=plt.cm.viridis,
                              alpha=0.8)
        
        nx.draw_networkx_edges(network_graph, pos, ax=ax1,
                              alpha=0.3, 
                              edge_color='gray',
                              width=0.5)
        
        # Ajouter des labels pour les nœuds les plus importants
        if len(network_graph.nodes()) < 50:
            nx.draw_networkx_labels(network_graph, pos, ax=ax1, font_size=6)
        else:
            # Afficher seulement les nœuds avec le plus haut degré
            top_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:10]
            top_nodes_dict = {node: str(node) for node, _ in top_nodes}
            nx.draw_networkx_labels(network_graph, pos, top_nodes_dict, ax=ax1, font_size=8, font_weight='bold')
        
        ax1.set_title(f'Topologie du Réseau\n({len(network_graph.nodes())} nœuds, {len(network_graph.edges())} arêtes)')
        ax1.axis('off')
        
        # 2. Distribution des degrés
        ax2 = axes[0, 1]
        degree_values = list(degrees.values())
        n_bins = min(30, max(10, len(set(degree_values))))
        
        ax2.hist(degree_values, bins=n_bins, alpha=0.7, color='skyblue', edgecolor='black')
        ax2.set_xlabel('Degré des nœuds')
        ax2.set_ylabel('Nombre de nœuds')
        ax2.set_title('Distribution des degrés')
        ax2.grid(True, alpha=0.3)
        ax2.set_yscale('log' if max(degree_values) > 100 else 'linear')
        
        # 3. Top nœuds par connectivité
        ax3 = axes[1, 0]
        top_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:15]
        
        if top_nodes:
            nodes, degs = zip(*top_nodes)
            y_pos = range(len(nodes))
            bars = ax3.barh(y_pos, degs, color='lightcoral', alpha=0.8)
            ax3.set_yticks(y_pos)
            ax3.set_yticklabels([str(n)[:30] + '...' if len(str(n)) > 30 else str(n) for n in nodes])
            ax3.set_xlabel('Degré de connectivité')
            ax3.set_title('Top 15 Nœuds les Plus Connectés')
            ax3.grid(True, alpha=0.3, axis='x')
            
            # Ajouter les valeurs sur les barres
            for i, bar in enumerate(bars):
                width = bar.get_width()
                ax3.text(width + max(degs) * 0.01, bar.get_y() + bar.get_height()/2, 
                        f'{int(width)}', ha='left', va='center', fontsize=8)
        
        # 4. Statistiques détaillées
        ax4 = axes[1, 1]
        
        # Calculer les statistiques
        avg_degree = np.mean(degree_values)
        median_degree = np.median(degree_values)
        max_degree = max(degree_values)
        min_degree = min(degree_values)
        density = nx.density(network_graph)
        
        # Connectivité
        try:
            if network_graph.is_directed():
                is_connected = nx.is_weakly_connected(network_graph)
                components = nx.number_weakly_connected_components(network_graph)
            else:
                is_connected = nx.is_connected(network_graph)
                components = nx.number_connected_components(network_graph)
        except:
            is_connected = "N/A"
            components = "N/A"
        
        # Diamètre (pour les graphes connexes et pas trop grands)
        diameter = "N/A"
        if is_connected == True and len(network_graph.nodes()) < 1000:
            try:
                diameter = nx.diameter(network_graph)
            except:
                diameter = "N/A"
        
        stats_text = f"""STATISTIQUES DU RÉSEAU
━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Structure:
   • Nœuds: {len(network_graph.nodes()):,}
   • Arêtes: {len(network_graph.edges()):,}
   • Type: {'Dirigé' if network_graph.is_directed() else 'Non dirigé'}
   • Densité: {density:.6f}

🔗 Connectivité:
   • Connexe: {is_connected}
   • Composantes: {components}
   • Diamètre: {diameter}

📈 Degrés:
   • Moyenne: {avg_degree:.2f}
   • Médiane: {median_degree:.2f}
   • Maximum: {max_degree}
   • Minimum: {min_degree}

🎯 Nœud le plus connecté:
   • {max(degrees, key=degrees.get)}
   • Degré: {max_degree}
        """
        
        ax4.text(0.05, 0.95, stats_text, transform=ax4.transAxes, 
                fontfamily='monospace', fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
        ax4.set_xlim(0, 1)
        ax4.set_ylim(0, 1)
        ax4.axis('off')
        ax4.set_title('Statistiques Détaillées')
        
        plt.tight_layout()
        plt.show()
        
        # Résumé en console
        print(f"\n🎉 VISUALISATION CRÉÉE AVEC SUCCÈS!")
        print(f"📊 Réseau analysé: {len(network_graph.nodes())} nœuds, {len(network_graph.edges())} arêtes")
        print(f"🔗 Degré moyen: {avg_degree:.2f}")
        print(f"⭐ Nœud le plus important: {max(degrees, key=degrees.get)} (degré: {max_degree})")
        
    else:
        print(f"❌ Impossible d'extraire un graphe NetworkX pour la visualisation")
        print(f"💡 L'objet contient probablement les données sous un format différent")
        
        # Essayer d'autres approches
        print(f"\n🔍 Tentative d'exploration alternative...")
        
        # Lister tous les attributs et méthodes
        all_attrs = [attr for attr in dir(topology_data) if not attr.startswith('_')]
        print(f"📋 Tous les attributs/méthodes: {all_attrs}")
        
        # Essayer d'appeler quelques méthodes sans paramètres
        for method_name in ['get_graph_stats', 'pending_requests']:
            if hasattr(topology_data, method_name):
                try:
                    result = getattr(topology_data, method_name)
                    if callable(result):
                        result = result()
                    print(f"✅ {method_name}: {type(result)} - {str(result)[:100]}...")
                except Exception as e:
                    print(f"❌ {method_name}: Erreur - {e}")

except Exception as e:
    print(f"❌ Erreur générale: {e}")
    print(f"🔍 Type d'erreur: {type(e).__name__}")

🔍 Exploration de l'objet NetworkTopologyGraph...
📊 Extraction du graphe NetworkX...
✅ Graphe extrait via get_graph()
Type du graphe: <class 'networkx.classes.multidigraph.MultiDiGraph'>
🔗 Graphe NetworkX détecté!
   📊 Nœuds: 40161
   📊 Arêtes: 907656

📈 Extraction des statistiques...
✅ Statistiques obtenues:
Type des stats: <class 'dict'>
Contenu: {'num_nodes': 40161, 'num_edges': 907656, 'central_nodes': 33705, 'leaf_nodes': 6456}

🔍 Exploration des attributs...
📝 Pending requests: <class 'dict'> - Taille: 0
🔎 Méthode find_node disponible
🔗 Méthode find_edge disponible

🎨 CRÉATION DE LA VISUALISATION...
🔄 Calcul des positions des nœuds...
